# Pipeline questionario: merge, analisi, selezione colonne
Questo notebook esegue in sequenza:
1) Unione dei file Excel e aggiunta del numero file
2) Analisi delle intestazioni e coperture
3) Selezione delle colonne utili e creazione dataset ridotto

In [2]:
# 1) Merge dei file Excel dalla cartella "dati backup" (sotto la cartella del notebook)
import os, pandas as pd, re

base_dir = os.getcwd()
input_dir = os.path.join(base_dir, "dati backup")
pattern = re.compile(r"results-survey(\d+)\.xlsx")

if not os.path.isdir(input_dir):
    raise FileNotFoundError(f"Cartella non trovata: {input_dir}")

dfs = []
for fn in os.listdir(input_dir):
    m = pattern.match(fn)
    if m:
        num = m.group(1)
        full_path = os.path.join(input_dir, fn)
        df = pd.read_excel(full_path)
        df["file_number"] = num
        dfs.append(df)

merged = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
merged.to_excel("merged_results.xlsx", index=False)
len(merged), merged.columns.tolist()[:10]

(5,
 ['ID risposta',
  'Data invio',
  'Ultima pagina',
  'Lingua iniziale',
  'Seme',
  '1.1 Ruolo:',
  '1.1 Ruolo: [Altro]',
  '1.2 Età:',
  '1.3 Genere:',
  '1.4 Titolo di studio (indichi tutti i titoli posseduti): [laurea ]'])

In [3]:
# 2) Analisi intestazioni e copertura
import pandas as pd, unicodedata, re
def remove_diacritics(s):
    nfkd = unicodedata.normalize('NFKD', str(s));
    return ''.join(ch for ch in nfkd if not unicodedata.combining(ch))
def normalize_name(s):
    s=str(s); s=remove_diacritics(s).lower().strip();
    s=re.sub(r'\s+',' ',s); s=re.sub(r'[^0-9a-z]','',s); return s
df = pd.read_excel('merged_results.xlsx')
rows = []
for c in df.columns:
    s = df[c]
    rows.append({
        'original_name': c,
        'normalized_name': normalize_name(c),
        'dtype': str(s.dtype),
        'row_count': len(df),
        'non_null_count': int(s.notna().sum()),
        'null_count': int(s.isna().sum()),
        'non_null_pct': round(100*s.notna().mean(),2),
        'unique_count_non_null': int(s.dropna().nunique()) if s.notna().any() else 0,
        'sample_values': ', '.join(s.dropna().astype(str).unique()[:3])
    })
ha = pd.DataFrame(rows)
ha.to_csv('header_analysis.csv', index=False)
# copertura per file
if 'file_number' in df.columns:
    long = []
    for fn, g in df.groupby('file_number', dropna=False):
        n=len(g)
        for c in df.columns:
            if c=='file_number': continue
            non_null=int(g[c].notna().sum()); pct=round(100*non_null/max(n,1),2)
            long.append({'file_number':fn,'column':c,'non_null_pct':pct,'non_null_count':non_null,'rows_in_file':n})
    pd.DataFrame(long).to_csv('header_coverage_by_file.csv', index=False)
ha.head(8)

,original_name,normalized_name,dtype,row_count,non_null_count,null_count,non_null_pct,unique_count_non_null,sample_values
0,ID risposta,idrisposta,int64,5,5,0,100.0,5,"1, 2, 3"
1,Data invio,datainvio,object,5,2,3,40.0,1,1980-01-01 00:00:00
2,Ultima pagina,ultimapagina,float64,5,4,1,80.0,3,"26.0, 42.0, 48.0"
3,Lingua iniziale,linguainiziale,object,5,5,0,100.0,1,it
4,Seme,seme,int64,5,5,0,100.0,5,"1102216992, 1266523766, 1011537043"
5,1.1 Ruolo:,11ruolo,object,5,4,1,80.0,2,"docente, Altro"
6,1.1 Ruolo: [Altro],11ruoloaltro,object,5,1,4,20.0,1,A022
7,1.2 Età:,12eta,object,5,4,1,80.0,1,più di 60


In [4]:
# 3) Selezione colonne utili e dataset ridotto
import pandas as pd, re
OPEN_TEXT_KEYWORDS = [
    'Specificare', 'Quali?', 'Quali ', 'Scriva', 'A quali', 'Indichi i corsi',
    'titolo del corso', 'ente organizzatore', 'tipologia (master', 'durata (indicare in ore)',
    'anno di conseguimento', 'Altro]'
]
META_EXACT = { 'ID risposta','Data invio','Ultima pagina','Lingua iniziale','Seme','Data di inizio','Data dellultima azione','file_number'}
EXCLUDE_PREFIXES = ['Tempo totale','Tempo per il gruppo di domande','Tempo per la domanda']
TITLE_OF_STUDY_WHITELIST = [
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [diploma]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [laurea]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di perfezionamento ]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di specializzazione ]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [master ]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [dottorato ]',
]
KEEP_PREFIXES = ['1.','2.','3.','4.','5.','6.','7.']
KEEP_STRICT_PREFIXES = ['3.17 ']
ha = pd.read_csv('header_analysis.csv')
candidates = []
def is_open_text(name:str)->bool:
    nl=name.lower();
    return any(k.lower() in nl for k in OPEN_TEXT_KEYWORDS)
for _,row in ha.iterrows():
    name = str(row['original_name'])
    if name in META_EXACT or any(name.startswith(p) for p in EXCLUDE_PREFIXES):
        continue
    non_null = int(row.get('non_null_count',0))
    unique_cnt = int(row.get('unique_count_non_null',0)) if pd.notna(row.get('unique_count_non_null',0)) else 0
    if non_null==0: continue
    if is_open_text(name) and unique_cnt>20: continue
    if name in TITLE_OF_STUDY_WHITELIST: candidates.append(name); continue
    if any(name.startswith(p) for p in KEEP_STRICT_PREFIXES): candidates.append(name); continue
    if any(name.startswith(p) for p in KEEP_PREFIXES):
        if is_open_text(name) and unique_cnt>20: continue
        candidates.append(name)
keep=[]; seen=set()
for c in candidates:
    if c not in seen:
        keep.append(c); seen.add(c)
subset = pd.read_excel('merged_results.xlsx')[ [c for c in keep if c in pd.read_excel('merged_results.xlsx').columns] ]
subset.to_csv('dataset_domande.csv', index=False)

print(f"Totale domande selezionate: {len(keep)}")
print("\n=== TUTTE LE DOMANDE SELEZIONATE ===")
for i, question in enumerate(keep, 1):
    print(f"{i:3d}. {question}")

Totale domande selezionate: 181

=== TUTTE LE DOMANDE SELEZIONATE ===
  1. 1.1 Ruolo:
  2. 1.1 Ruolo: [Altro]
  3. 1.2 Età:
  4. 1.3 Genere:
  5. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [laurea ]
  6. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di perfezionamento ]
  7. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di specializzazione ]
  8. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [master ]
  9. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [dottorato ]
 10. 1.5 Anzianità di servizio non di ruolo nella scuola:
 11. 1.6 Anzianità di servizio di ruolo nella scuola:
 12. 1.7 Attualmente svolge altri incarichi (vicepreside, funzione strumentale etc.)? 
 13. 1.7.b Specificare:
 14. 1.8 Per quale classe di concorso ha conseguito l’abilitazione all’insegnamento? Se ha conseguito l’abilitazione per più classi di concorso, le indichi separandole con una virgola (es. A012,A013)
 15. 1.9 Insegnamento in qualità di 

In [4]:
# 4) Analisi domande: raggruppamento e rilevazione Likert (senza grafici)
import re, pandas as pd, unicodedata

# 4.1 Carica dataset (se non esiste già in memoria)
try:
    _ = data  # verifica esistenza
except NameError:
    try:
        data = pd.read_csv('dataset_domande.csv')
    except FileNotFoundError:
        data = pd.read_excel('merged_results.xlsx')

# 4.2 Utility testo

def strip_accents(s: str) -> str:
    if s is None:
        return ''
    s = str(s)
    nfkd = unicodedata.normalize('NFKD', s)
    return ''.join(ch for ch in nfkd if not unicodedata.combining(ch))

def norm_txt(s: str) -> str:
    s = strip_accents(s).lower().strip()
    s = re.sub(r'\s+', ' ', s)
    return s

# 4.3 Raggruppa colonne per prefisso numerico (es. '2.3')
num_pat = re.compile(r'^(\d+\.\d+)(?:[\s\S]*)$')
question_groups = {}
for col in data.columns:
    m = num_pat.match(col)
    if m:
        key = m.group(1)
        question_groups.setdefault(key, []).append(col)

# 4.4 Etichetta leggibile domanda (numero — testo)
clean_q_text_pat_prefix = re.compile(r'^\d+\.\d+\s*')
clean_q_text_pat_brackets = re.compile(r'\s*\[[^\]]+\]\s*$')

def clean_question_text(col: str) -> str:
    s = str(col)
    s = clean_q_text_pat_prefix.sub('', s).strip()
    s = clean_q_text_pat_brackets.sub('', s).strip()
    s = s.rstrip(':').strip()
    return s

group_labels = {}
for key, cols in question_groups.items():
    texts = [clean_question_text(c) for c in cols if c]
    group_labels[key] = max(set(texts), key=lambda t: (texts.count(t), len(t))) if texts else key

# 4.5 Definizione famiglie Likert e token mapping
LIKERT_FAMILIES = {
    'intensita': {
        'order': ['Per nulla', 'Poco', 'Abbastanza', 'Molto', 'Moltissimo'],
        'tokens': {
            'per nulla': 'Per nulla', 'per niente': 'Per nulla', 'niente affatto': 'Per nulla',
            'poco': 'Poco',
            'abbastanza': 'Abbastanza', 'sufficientemente': 'Abbastanza', 'mediamente': 'Abbastanza',
            'molto': 'Molto', 'tanto': 'Molto',
            'moltissimo': 'Moltissimo', 'estremamente': 'Moltissimo'
        }
    },
    'accordo': {
        'order': ["Per nulla d'accordo", "In disaccordo", 'Neutrale', "D'accordo", "Molto d'accordo"],
        'tokens': {
            "per nulla d'accordo": "Per nulla d'accordo", 'fortemente in disaccordo': "Per nulla d'accordo",
            'in disaccordo': 'In disaccordo',
            'neutrale': 'Neutrale', "ne d'accordo ne in disaccordo": 'Neutrale', 'indifferente': 'Neutrale',
            "d'accordo": "D'accordo",
            "molto d'accordo": "Molto d'accordo", "completamente d'accordo": "Molto d'accordo"
        }
    },
    'frequenza': {
        'order': ['Mai', 'Raramente', 'A volte', 'Spesso', 'Sempre'],
        'tokens': {
            'mai': 'Mai',
            'raramente': 'Raramente', 'poco spesso': 'Raramente',
            'a volte': 'A volte', 'talvolta': 'A volte', 'occasionalmente': 'A volte',
            'spesso': 'Spesso', 'frequentemente': 'Spesso',
            'sempre': 'Sempre', 'quasi sempre': 'Sempre'
        }
    },
    'qualita': {
        'order': ['Insufficiente', 'Sufficiente', 'Buona', 'Ottima'],
        'tokens': {
            'insufficiente': 'Insufficiente', 'scarsa': 'Insufficiente', 'pessima': 'Insufficiente',
            'sufficiente': 'Sufficiente', 'discreta': 'Sufficiente',
            'buona': 'Buona',
            'ottima': 'Ottima', 'eccellente': 'Ottima'
        }
    }
}

# 4.6 Helper per rilevazione Likert e mapping

def first_non_na(series: pd.Series):
    for v in series:
        if pd.notna(v) and str(v).strip() != '':
            return v
    return None

def guess_family_from_first_row(cols):
    fam_counts = {}
    for c in cols:
        v = first_non_na(data[c])
        if v is None:
            continue
        nv = norm_txt(str(v))
        matched_family = None
        for fam, cfg in LIKERT_FAMILIES.items():
            for token in cfg['tokens'].keys():
                if token in nv:
                    matched_family = fam
                    break
            if matched_family:
                break
        if matched_family:
            fam_counts[matched_family] = fam_counts.get(matched_family, 0) + 1
    return max(fam_counts, key=fam_counts.get) if fam_counts else None

def map_value_to_canonical(v: str, family: str):
    if pd.isna(v) or str(v).strip() == '':
        return None
    nv = norm_txt(str(v))
    for token, canon in LIKERT_FAMILIES[family]['tokens'].items():
        if token in nv:
            return canon
    return None

# 4.7 Rileva famiglia Likert per ogni gruppo
_group_families = {g: guess_family_from_first_row(cols) for g, cols in question_groups.items()}
likert_summary = pd.DataFrame([
    {
        'group': g,
        'label': group_labels.get(g, g),
        'family': _group_families[g] if _group_families[g] else 'non-likert',
        'n_cols': len(question_groups[g])
    }
    for g in sorted(question_groups.keys(), key=lambda k: (int(k.split('.')[0]), int(k.split('.')[1])))
])

# Mostra un estratto della sintesi
likert_summary.head(200)

,group,label,family,n_cols
0,1.1,Ruolo,non-likert,2
1,1.2,Età,non-likert,1
2,1.3,Genere,non-likert,1
3,1.4,Titolo di studio (indichi tutti i titoli posse...,non-likert,7
4,1.5,Anzianità di servizio non di ruolo nella scuola,non-likert,1
5,1.6,Anzianità di servizio di ruolo nella scuola,non-likert,1
6,1.7,Attualmente svolge altri incarichi (vicepresid...,non-likert,1
7,1.8,Per quale classe di concorso ha conseguito l’a...,non-likert,1
8,1.9,Insegnamento in qualità di docente per le atti...,non-likert,1
9,1.10,Tipologia dell’istituto scolastico presso cui ...,non-likert,1


In [5]:
#  [Diagnostica] Ambiente e renderer Plotly
import sys, plotly
import plotly.io as pio
from packaging import version

# Rileva versione nbformat se presente e compatibile
try:
    import nbformat  # noqa: F401
    nbv = nbformat.__version__
    has_compatible_nbformat = version.parse(nbv) >= version.parse("4.2.0")
except Exception as e:
    nbv = f"missing ({e})"
    has_compatible_nbformat = False

# Funzione robusta per impostare un renderer sicuro in questo ambiente
def _configure_plotly_renderer():
    available = set(pio.renderers)  # renderers disponibili in questa sessione
    # Preferenze in ordine: plotly_mimetype (se nbformat>=4.2.0), vscode, notebook_connected, browser
    if has_compatible_nbformat and 'plotly_mimetype' in available:
        pio.renderers.default = 'plotly_mimetype'
        return
    if 'vscode' in available:
        pio.renderers.default = 'vscode'
        return
    if 'notebook_connected' in available:
        pio.renderers.default = 'notebook_connected'
        return
    # Fallback universale: apre nel browser di sistema
    pio.renderers.default = 'browser'

_configure_plotly_renderer()

print("Python:", sys.version.split()[0])
print("Executable:", sys.executable)
print("Plotly:", plotly.__version__)
print("nbformat:", nbv)
print("nbformat compatible:", has_compatible_nbformat)
print("Renderer:", pio.renderers.default)

Python: 3.9.19
Executable: /Users/desi76/repo-git-nugh/secondo grado/.venv/bin/python
Plotly: 6.3.0
nbformat: 5.10.4
nbformat compatible: True
Renderer: plotly_mimetype


In [58]:
# 🎯 ANALISI COMPLETA SENZA WIDGET
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import numpy as np
import textwrap
try:
    import scipy.stats as stats
except ImportError:
    print("⚠️  scipy non disponibile - grafici gaussiani limitati")
    stats = None

def wrap_title(title, max_chars=140):
    """
    Fa andare a capo i titoli lunghi per i grafici
    """
    if len(title) <= max_chars:
        return title
    
    # Usa textwrap per dividere il testo in righe
    wrapped = textwrap.fill(title, width=max_chars)
    return wrapped.replace('\n', '<br>')  # Plotly usa <br> per andare a capo

def analyze_question_group(group_key='2.4', chart_type='bar', show_percentages=True, include_na=False):
    """
    Analizza un gruppo di domande con statistiche complete e tipi di grafico
    
    Parametri:
    - group_key: chiave del gruppo (es. '2.4')
    - chart_type: 'bar', 'bar_h', 'pie', 'likert_bar', 'histogram', 'gaussian'
    - show_percentages: True per %, False per numeri assoluti (quando applicabile)
    - include_na: include valori NA nell'analisi
    """
    print(f"🎯 ANALISI DOMANDA: {group_key}")
    print(f"📝 Descrizione: {group_labels.get(group_key, group_key)}")
    print(f"📊 Tipo grafico: {chart_type}")
    print(f"📈 Valori: {'Percentuali' if show_percentages else 'Numeri assoluti'}")
    print(f"🔢 Includi NA: {include_na}")
    
    # Spiegazione del tipo di grafico
    chart_explanations = {
        'bar': "📊 Barre verticali - Ideale per confrontare categorie, facile da leggere",
        'bar_h': "📊 Barre orizzontali - Migliore quando le etichette sono lunghe",
        'pie': "🥧 Grafico a torta - Mostra proporzioni del totale, max 5-6 categorie",
        'likert_bar': "📈 Barre Likert - Ordinate secondo scala (Per nulla → Molto)",
        'histogram': "📉 Istogramma - Mostra distribuzione di valori numerici",
        'gaussian': "🔔 Curva gaussiana - Istogramma + curva normale sovrapposta"
    }
    
    if chart_type in chart_explanations:
        print(f"💡 {chart_explanations[chart_type]}")
    print()
    
    if group_key not in question_groups:
        print(f"❌ Gruppo {group_key} non trovato")
        return
    
    cols = question_groups[group_key]
    print(f"🔢 Numero di sotto-domande: {len(cols)}")
    print("=" * 80)
    
    # Per ogni colonna del gruppo, crea un grafico
    for i, col in enumerate(cols, 1):
        print(f"📊 Sotto-domanda {i}:")
        print(f"    {col}")
        
        # Prendi i dati della colonna
        series = data[col]
        original_count = len(series)
        
        if not include_na:
            series = series.dropna()
        
        if len(series) == 0:
            print("   ⚠️  Nessun dato disponibile")
            continue
            
        # Conta i valori
        counts = Counter(series)
        total = sum(counts.values())
        
        # Statistiche descrittive
        print(f"   📈 Statistiche:")
        print(f"      • Risposte totali: {total}")
        if not include_na and original_count > total:
            print(f"      • Valori mancanti: {original_count - total}")
        
        # Calcola statistiche numeriche se applicabile
        try:
            # Prova a mappare i valori Likert a numeri per le statistiche
            likert_family = _group_families.get(group_key)
            if likert_family and likert_family in LIKERT_FAMILIES:
                order = LIKERT_FAMILIES[likert_family]['order']
                # Mappa i valori a numeri (1, 2, 3, ...)
                numeric_values = []
                for val in series:
                    try:
                        idx = order.index(val)
                        numeric_values.append(idx + 1)
                    except ValueError:
                        continue
                
                if numeric_values:
                    mean_val = np.mean(numeric_values)
                    median_val = np.median(numeric_values)
                    std_val = np.std(numeric_values, ddof=1) if len(numeric_values) > 1 else 0
                    
                    print(f"      • Media: {mean_val:.2f}")
                    print(f"      • Mediana: {median_val:.1f}")
                    print(f"      • Deviazione standard: {std_val:.2f}")
        except Exception:
            pass
        
        # Distribuzione dei valori
        print(f"   📊 Distribuzione:")
        for value, count in counts.most_common():
            pct = round(100 * count / total, 1) if total > 0 else 0
            print(f"      • {value}: {count} ({pct}%)")
        
        # Crea grafico in base al tipo richiesto
        if len(counts) > 0:
            labels = list(counts.keys())
            values = list(counts.values())
            
            # Palette di colori vivaci per tutti i grafici
            colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', 
                     '#FF9FF3', '#54A0FF', '#5F27CD', '#00D2D3', '#FF9F43',
                     '#6C5CE7', '#A29BFE', '#FD79A8', '#E17055', '#00B894']
            
            # Titolo con a capo automatico
            wrapped_title = wrap_title(col)
            
            # Calcola valori per i grafici - sempre mostra entrambi
            percentages = [round(100 * v / total, 1) for v in values]
            # Testo che mostra sia numero assoluto che percentuale
            text_labels = [f"{v} ({p}%)" for v, p in zip(values, percentages)]
            
            # Display values per l'asse Y
            display_values = percentages if show_percentages else values
            y_label = 'Percentuale (%)' if show_percentages else 'Conteggio'
            
            if chart_type == 'bar':
                # Grafico a barre verticali
                fig = px.bar(
                    x=labels, 
                    y=display_values,
                    title=wrapped_title,
                    labels={'x': 'Risposta', 'y': y_label},
                    text=text_labels,
                    color=labels,
                    color_discrete_sequence=colors
                )
            elif chart_type == 'bar_h':
                # Grafico a barre orizzontali
                fig = px.bar(
                    x=display_values,
                    y=labels,
                    title=wrapped_title,
                    labels={'x': y_label, 'y': 'Risposta'},
                    orientation='h',
                    text=text_labels,
                    color=labels,
                    color_discrete_sequence=colors
                )
            elif chart_type == 'pie':
                # Grafico a torta con etichette personalizzate
                fig = px.pie(
                    names=labels,
                    values=values,  # Sempre valori assoluti per le torte
                    title=wrapped_title,
                    color_discrete_sequence=colors
                )
                # Aggiunge etichette personalizzate alla torta
                fig.update_traces(textinfo='label+value+percent', 
                                texttemplate='%{label}<br>%{value} (%{percent})')
            elif chart_type == 'likert_bar':
                # Grafico Likert ordinato
                likert_family = _group_families.get(group_key)
                if likert_family and likert_family in LIKERT_FAMILIES:
                    order = LIKERT_FAMILIES[likert_family]['order']
                    ordered_labels = [l for l in order if l in labels]
                    ordered_values = [counts[l] for l in ordered_labels]
                    ordered_display = [round(100 * v / total, 1) for v in ordered_values] if show_percentages else ordered_values
                    ordered_text = [f"{v} ({round(100 * v / total, 1)}%)" for v in ordered_values]
                    
                    fig = px.bar(
                        x=ordered_labels, 
                        y=ordered_display,
                        title=wrapped_title,
                        labels={'x': 'Risposta', 'y': y_label},
                        text=ordered_text,
                        color=ordered_labels,
                        color_discrete_sequence=colors
                    )
                else:
                    # Fallback a barre normali se non è Likert
                    fig = px.bar(x=labels, y=display_values, title=wrapped_title, 
                               labels={'x': 'Risposta', 'y': y_label}, 
                               text=text_labels,
                               color=labels, color_discrete_sequence=colors)
            elif chart_type == 'histogram':
                # Istogramma (per dati numerici Likert)
                likert_family = _group_families.get(group_key)
                if likert_family and likert_family in LIKERT_FAMILIES:
                    order = LIKERT_FAMILIES[likert_family]['order']
                    numeric_data = []
                    for val in series:
                        try:
                            idx = order.index(val)
                            numeric_data.extend([idx + 1] * 1)  # Mappa a numeri 1,2,3...
                        except ValueError:
                            continue
                    
                    if numeric_data:
                        fig = px.histogram(
                            x=numeric_data,
                            title=wrapped_title,
                            labels={'x': 'Valore Likert (1=Per nulla, 5=Moltissimo)', 'y': 'Frequenza'},
                            nbins=len(order),
                            color_discrete_sequence=[colors[0]]
                        )
                    else:
                        fig = px.bar(x=labels, y=display_values, title=wrapped_title,
                                   text=text_labels,
                                   color=labels, color_discrete_sequence=colors)
                else:
                    # Per dati non-Likert, usa grafico a barre normale
                    fig = px.bar(x=labels, y=display_values, title=wrapped_title, 
                               text=text_labels,
                               color=labels, color_discrete_sequence=colors)
            elif chart_type == 'gaussian':
                # Curva gaussiana sovrapposta
                likert_family = _group_families.get(group_key)
                if likert_family and likert_family in LIKERT_FAMILIES:
                    order = LIKERT_FAMILIES[likert_family]['order']
                    numeric_data = []
                    for val in series:
                        try:
                            idx = order.index(val)
                            numeric_data.append(idx + 1)
                        except ValueError:
                            continue
                    
                    if numeric_data and len(numeric_data) > 1:
                        import scipy.stats as stats
                        
                        # Crea istogramma
                        fig = px.histogram(
                            x=numeric_data,
                            title=wrapped_title,
                            labels={'x': 'Valore Likert', 'y': 'Frequenza'},
                            nbins=len(order),
                            opacity=0.7,
                            color_discrete_sequence=[colors[0]]
                        )
                        
                        # Aggiunge curva gaussiana
                        x_range = np.linspace(min(numeric_data), max(numeric_data), 100)
                        mean_val = np.mean(numeric_data)
                        std_val = np.std(numeric_data)
                        gaussian_curve = stats.norm.pdf(x_range, mean_val, std_val)
                        # Scala la curva per matchare l'istogramma
                        gaussian_curve = gaussian_curve * len(numeric_data) * (max(numeric_data) - min(numeric_data)) / len(order)
                        
                        fig.add_scatter(
                            x=x_range, 
                            y=gaussian_curve,
                            mode='lines',
                            name='Curva Normale',
                            line=dict(color='red', width=3)
                        )
                    else:
                        fig = px.bar(x=labels, y=display_values, title=wrapped_title,
                                   text=text_labels,
                                   color=labels, color_discrete_sequence=colors)
                else:
                    fig = px.bar(x=labels, y=display_values, title=wrapped_title, 
                               text=text_labels,
                               color=labels, color_discrete_sequence=colors)
            else:
                # Default: barre verticali
                fig = px.bar(
                    x=labels, 
                    y=display_values,
                    title=wrapped_title,
                    labels={'x': 'Risposta', 'y': y_label},
                    text=text_labels,
                    color=labels,
                    color_discrete_sequence=colors
                )
            
            # Styling avanzato con margine superiore dinamico per titoli lunghi
            title_lines = wrapped_title.count('<br>') + 1
            top_margin = 100 + (title_lines - 1) * 30  # Margine superiore più ampio
            
            # Calcola margine sinistro dinamico per etichette lunghe
            max_label_length = max(len(str(label)) for label in labels) if labels else 0
            left_margin = max(80, min(200, 80 + max_label_length * 3))  # Margine sinistro adattivo
            
            # Altezza adattiva basata sul numero di elementi
            adaptive_height = max(500, 400 + len(labels) * 20)  # Altezza minima 500px
            
            fig.update_layout(
                xaxis_tickangle=-45,
                height=adaptive_height,
                margin={
                    'l': left_margin, 
                    'r': 80,  # Margine destro più ampio
                    't': top_margin, 
                    'b': 150  # Margine inferiore più ampio per etichette inclinate
                },
                plot_bgcolor='rgba(240,240,245,0.8)',
                paper_bgcolor='white',
                font=dict(size=12),
                title_font=dict(size=14, color='#2c3e50'),
                showlegend=False,  # Nasconde leggenda per tutti i grafici tranne torte
                xaxis=dict(
                    title_font=dict(size=12, color='#34495e'),
                    tickfont=dict(size=10, color='#2c3e50'),
                    gridcolor='rgba(200,200,200,0.3)',
                    automargin=True  # Margine automatico per evitare tagli
                ) if chart_type != 'pie' else {},
                yaxis=dict(
                    title_font=dict(size=12, color='#34495e'),
                    tickfont=dict(size=10, color='#2c3e50'),
                    gridcolor='rgba(200,200,200,0.3)',
                    automargin=True  # Margine automatico per evitare tagli
                ) if chart_type != 'pie' else {}
            )
            
            # Per grafici a torta, mostra la leggenda se ci sono molte categorie
            if chart_type == 'pie':
                fig.update_layout(showlegend=(len(labels) > 4))
            
            # Aggiunge dettagli alle barre (eccetto torta)
            if chart_type != 'pie':
                fig.update_traces(
                    marker_line_color='white',
                    marker_line_width=2,
                    opacity=0.85,
                    texttemplate='%{text}',  # Usa il testo personalizzato
                    textposition='outside',
                    textfont_size=11,
                    textfont_color='#2c3e50',
                    hovertemplate='<b>%{x}</b><br>%{text}<br><extra></extra>'  # Tooltip pulito
                )
            else:
                # Per grafici a torta, personalizza il tooltip
                fig.update_traces(
                    hovertemplate='<b>%{label}</b><br>%{value} (%{percent})<extra></extra>'
                )
            
            fig.show()
        
        print("-" * 50)
    
    print("=" * 80)

# Funzioni di supporto per diversi tipi di analisi
def analyze_bar(group_key, percentages=True, include_na=False):
    """Analisi con grafici a barre verticali"""
    return analyze_question_group(group_key, 'bar', percentages, include_na)

def analyze_bar_h(group_key, percentages=True, include_na=False):
    """Analisi con grafici a barre orizzontali"""
    return analyze_question_group(group_key, 'bar_h', percentages, include_na)

def analyze_pie(group_key, include_na=False):
    """Analisi con grafici a torta"""
    return analyze_question_group(group_key, 'pie', True, include_na)

def analyze_likert(group_key, percentages=True, include_na=False):
    """Analisi con grafici Likert ordinati"""
    return analyze_question_group(group_key, 'likert_bar', percentages, include_na)

def analyze_histogram(group_key, include_na=False):
    """Analisi con istogramma (per dati Likert numerici)"""
    return analyze_question_group(group_key, 'histogram', True, include_na)

def analyze_gaussian(group_key, include_na=False):
    """Analisi con curva gaussiana sovrapposta"""
    return analyze_question_group(group_key, 'gaussian', True, include_na)

# 🎯 Funzioni di analisi pronte!
print("🎯 Funzioni di analisi pronte!")
print()
print("📊 Uso completo:")
print("   analyze_question_group('2.4', 'bar', True, False)")
print("   → ('domanda', 'tipo_grafico', percentuali?, includi_NA?)")
print()
print("📈 Tipi di grafico:")
print("   'bar'         → Barre verticali")
print("   'bar_h'       → Barre orizzontali") 
print("   'pie'         → Grafici a torta")
print("   'likert_bar'  → Barre Likert ordinate")
print("   'histogram'   → Istogramma") 
print("   'gaussian'    → Curva gaussiana")
print()
print("🚀 Funzioni rapide:")
print("   analyze_bar('2.4')        # Barre verticali %")
print("   analyze_bar_h('2.4')      # Barre orizzontali %") 
print("   analyze_pie('2.4')        # Torte")
print("   analyze_likert('2.4')     # Likert %")
print("   analyze_histogram('2.4')  # Istogramma")
print("   analyze_gaussian('2.4')   # Curva normale")
print()
print("💡 Per numeri assoluti: analyze_bar('2.4', percentages=False)")
print()
print(f"📋 Gruppi disponibili: {sorted(question_groups.keys())[:10]}...")

🎯 Funzioni di analisi pronte!

📊 Uso completo:
   analyze_question_group('2.4', 'bar', True, False)
   → ('domanda', 'tipo_grafico', percentuali?, includi_NA?)

📈 Tipi di grafico:
   'bar'         → Barre verticali
   'bar_h'       → Barre orizzontali
   'pie'         → Grafici a torta
   'likert_bar'  → Barre Likert ordinate
   'histogram'   → Istogramma
   'gaussian'    → Curva gaussiana

🚀 Funzioni rapide:
   analyze_bar('2.4')        # Barre verticali %
   analyze_bar_h('2.4')      # Barre orizzontali %
   analyze_pie('2.4')        # Torte
   analyze_likert('2.4')     # Likert %
   analyze_histogram('2.4')  # Istogramma
   analyze_gaussian('2.4')   # Curva normale

💡 Per numeri assoluti: analyze_bar('2.4', percentages=False)

📋 Gruppi disponibili: ['1.1', '1.10', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9']...


In [59]:
# 🧪 TEST: Analisi completa con statistiche
# Cambia questi parametri per testare diversi tipi di analisi:

GROUP = '1.1'           # Quale domanda analizzare
CHART_TYPE = 'bar'      # Tipo grafico (vedi spiegazioni sotto)
SHOW_PERCENTAGES = True # True = percentuali, False = numeri assoluti
INCLUDE_NA = False      # Includere valori mancanti

# 📊 TIPI DI GRAFICO DISPONIBILI:
# 'bar'         → Barre verticali (più leggibile)
# 'bar_h'       → Barre orizzontali (per etichette lunghe)  
# 'pie'         → Torta (buono per poche categorie)
# 'likert_bar'  → Barre Likert ordinate (Per nulla → Molto)
# 'histogram'   → Istogramma per distribuzioni numeriche
# 'gaussian'    → Curva gaussiana sovrapposta all'istogramma

# Esegui analisi
analyze_question_group(GROUP, CHART_TYPE, SHOW_PERCENTAGES, INCLUDE_NA)

🎯 ANALISI DOMANDA: 1.1
📝 Descrizione: Ruolo
📊 Tipo grafico: bar
📈 Valori: Percentuali
🔢 Includi NA: False
💡 📊 Barre verticali - Ideale per confrontare categorie, facile da leggere

🔢 Numero di sotto-domande: 2
📊 Sotto-domanda 1:
    1.1 Ruolo:
   📈 Statistiche:
      • Risposte totali: 169
   📊 Distribuzione:
      • docente orientatore: 120 (71.0%)
      • docente tutor: 24 (14.2%)
      • Altro: 14 (8.3%)
      • dirigente scolastico: 11 (6.5%)


--------------------------------------------------
📊 Sotto-domanda 2:
    1.1 Ruolo: [Altro]
   📈 Statistiche:
      • Risposte totali: 14
      • Valori mancanti: 155
   📊 Distribuzione:
      • Funzione strumentale orientamento: 1 (7.1%)
      • Referente Invalsi e RAV: 1 (7.1%)
      • Funzione strumentale Orientamento: 1 (7.1%)
      • gestore scuola: 1 (7.1%)
      • figura strumentale orientamento : 1 (7.1%)
      • Responsabile orientamento indirizzo turistico: 1 (7.1%)
      • Coordinatrice didattica e docente orientatore: 1 (7.1%)
      • Referente orientamento in uscita: 1 (7.1%)
      • Vice dirigente : 1 (7.1%)
      • Funzione strumentale per l’orientamento in ingresso: 1 (7.1%)
      • Docente : 1 (7.1%)
      • Referente per l'Orientamento in Uscita: 1 (7.1%)
      • Tutor ed orientatore in uscita: 1 (7.1%)
      • F.S. Referente per l'Orientamento: 1 (7.1%)


--------------------------------------------------


In [ ]:
# 🎨 ESEMPI DI TUTTI I TIPI DI GRAFICO
# Decommmenta quello che vuoi testare:

# analyze_question_group('2.4', 'bar', True, False)          # Barre verticali %
# analyze_question_group('2.4', 'bar', False, False)         # Barre verticali numeri assoluti
# analyze_question_group('2.4', 'bar_h', True, False)        # Barre orizzontali %  
# analyze_question_group('2.4', 'pie', True, False)          # Torta
# analyze_question_group('2.4', 'likert_bar', True, False)   # Likert ordinato %
# analyze_question_group('2.4', 'histogram', True, False)    # Istogramma 
# analyze_question_group('2.4', 'gaussian', True, False)     # Curva gaussiana

# 🚀 Oppure usa le funzioni rapide:
# analyze_gaussian('2.4')   # Prova questo per vedere la curva normale!

print("✅ Esempi pronti! Decommenta una riga per testare i diversi grafici")